
<center>



<h1> ADVANCED COMPUTING FOR ASTRONOMY LIBRARY (ACALib) </h1>


<b>Mauricio Araya, Martín Villanueva, Camilo Valenzuela, Axel Simonsen, Marcelo Jara, Teodoro Hochfarber, Manuel Sanchez, Gonzalo Moya.</b>

<img src="logoCHIVO-h260.png" />

</center>

# Repository Contributors

<img src="contrib.png"/>





# What is ACALib


## Facts:
* A ChiVO library written in Python <img src="python.png" width=200px />
* Uses the results of the research of past FONDEF and ICHAA projects <img src="fondef.png" width=200px />
* Focuses on *simple use of advanced computing* for astronomy <img src="acomp.jpg" width=200px />

## With *advanced computing* we refer to:
* Modern and algorithmically efficient astronomical algorithms
* VO-compatible in terms of data access and application layers,
* Parallel support for high-performance computing (e.g., MPI).




## Loading the library

**Goal:** Only one import 

In [1]:
%matplotlib inline
import sys
sys.path.append('../')

from acalib import *


SyntaxError: invalid syntax (container.py, line 16)

# Data in ACALib

**Goal:** Magical data loading
* FITS (yes) 
* VoTable (partially)
* MS (not yet)
* HDF5 (not yet)
* ASDF, ASDM? 

## Astronomy file formats are basically *containers* of:
1. Tables + Metadata
2. N-dimensional Images + Metadata
3. *Optional*: Hierarchical relations

## Our container has
* A `primary` astropy image (NDData) 
* A `images` list of astropy n-dimensional images (NDData) 
* A `tables` list of astropy tables (Table)

Let us load the Orion methanol line


In [ ]:
binpath = '../bindata/fits/cubes/'

# Data from ALMA science verification 
orion_path = binpath + 'Orion.methanol.cbc.contsub.image.fits'
container = load_fits(orion_path)
orion     = container.primary

In [ ]:
# Check Data

# Visualization

**Goal**: Try to easily *preview* data, not to make paper-ready images.

## The Visualize Function

The function `visualize()` use the *best effort* approach to visualize almost any data, from 1D to 3D.

In [ ]:
# This import should be removed in some point
from acalib.io.graph import *
#visualize(orion)

### 3D contours

In [ ]:
#visualize(orion,contour=True)

### 2D Visualization
Here we compute the zeroth moment using the `moment0()` function:
$$ M_0 = \sum_i I_i $$
were $I_i$ is the image at the $i$'th frequency, and visualize the result. We can also add contours to it.

In [ ]:
m0 = moment0(orion)
print "ORIGINAL"
visualize(m0)
print "ORIGINAL + CONTOUR"
visualize(m0,contour=True)

## Missing data

ACALib support images with masks.
Let us also load an observation of the M100 galaxy in Band 3

In [ ]:
m100_path  = binpath + 'M100line.image.fits'
container = load_fits(m100_path)
M100      = container.primary
print "MOMENT 0"
visualize(moment0(M100))
#visualize(M100)

## Spectrum visualization
**Warning**: this functionality is very experimental yet!

Here we compute an spectrum using the `spectra()` function, giving the position and aperture in *pixels*

In [ ]:
M100.wcs.celestial.printwcs()

In [ ]:
pos       = [301,301]*u.pix
ape       = 10*u.pix
m100_line = spectra(M100,position=pos,aperture=ape)
visualize(m100_line)

# Data Manipulation

## Other Moments
* We can use `moment1()` to compute:
$$M_1 = \frac{\sum_i I_i v_i}{M_0} $$
were $v_i$ is the velocity value at the $i$'th frequency (given a rest frequency).
* We can use `moment2()` to compute:
$$M_2 = \sqrt{\frac{\sum_i I_i (v_i - M_1)^2}{M_0}} $$


In [ ]:
print "MOMENT 1"
visualize(moment1(orion))
print "MOMENT 2"
visualize(moment2(orion))

## Simple Denoising

We can compute the RMS using `estimate_rms()` and denoise using function `denoise()`. We can visualize the moments again.

In [ ]:
rms       = estimate_rms(orion)
den_orion = denoise(orion,threshold=1.5*rms)
print "MOMENT 0"
visualize(moment0(den_orion))
print "MOMENT 1"
visualize(moment1(den_orion))
print "MOMENT 2"
visualize(moment2(den_orion))

## The M100 example
With and without contours

In [ ]:
print "ORIGINAL"
visualize(moment0(M100))
print "ORIGINAL + CONTOUR"
visualize(moment0(M100),contour=True)
rms  = estimate_rms(M100)
den_M100 = denoise(M100,threshold=2.0*rms)
print "DENOISED + CONTOUR"
visualize(moment0(den_M100),contour=True)
#visualize(den_M100,contour=True)


## Functions parameters
Functions have several default parameters, but they can be changed.
For example, the WCS includes a rest frequency to compute the radial velocity.
We can change this for the first moment.

In [ ]:
rfreq = den_M100.wcs.wcs.restfrq*u.Hz
print "MOMENT 1 with rest frequency",rfreq.to(u.GHz)
visualize(moment1(den_M100))
myrf=114.65*u.GHz
print "MOMENT 1 with rest frequency",myrf
visualize(moment1(den_M100,restfrq=myrf))

## Cut and Save
We can extract a subcube easily

In [ ]:
M100.wcs.printwcs()

In [ ]:
lb=[5,250,250]
ub=[35,350,350]
M100cut = cut(den_M100,lower=lb,upper=ub)
visualize(moment0(M100cut))
visualize(M100cut,contour=True)

In [ ]:
new_cont = Container()
new_cont.primary=M100cut
new_cont.save_fits("M100cut.fits")

In [ ]:
other_cont = load_fits("M100cut.fits")
element  = other_cont.primary
#print o_cut.wcs.wcs
elementM0   = moment0(element)
visualize(elementM0)

# Synthetic Data
## Methanol Cloud

In [ ]:
from acalib.synthetic import *

center   = [186.3,15.3]*u.deg
temp     = 300*u.K
rad_vel  = 150*u.km/u.s

univ=Universe()
univ.create_source('Methcloud',center)

# Defines a new component
mol_list = {'33SO2':[1.0,10.0]* u.Jy/u.beam}
offset   = [0,0]*u.arcsec
std      = [10,7]*u.arcsec
angle    = np.pi/9.*u.rad
fwhm     = 30*u.km/u.s
gradient = [0.0,0.0]*u.km/(u.s*u.arcsec)

# Create Component
model=GaussianIMC(mol_list,temp,offset,std,angle,fwhm,gradient)
model.set_velocity(rad_vel)
univ.add_component('Methcloud',model)


In [ ]:
# Create Cube
ang_res = [3.0,3.0]*u.arcsec
fov     = [200,200]*u.arcsec
freq    = 299.898*u.GHz
spe_res = 0.002*u.GHz
bw      = 0.2*u.GHz
noise   = 0.001*u.Jy/u.beam

cont = univ.gen_cube(center, ang_res, fov, freq, spe_res, bw, noise,noise/50.0)

## Visualize the Gaussian

In [ ]:
synthetic=cont.primary
print "ORIGINAL M0"
visualize(moment0(synthetic))
#visualize(synthetic)

In [ ]:
# Defines a new component
mol_list = {'33SO2': [0.5,5.0]* u.Jy/u.beam}
offset   = [20,-20]*u.arcsec
std      = [4,12]*u.arcsec
angle    = np.pi/4.*u.rad
fwhm     = 10*u.km/u.s
gradient = [-1.0,1.0]*u.km/(u.s*u.arcsec)
# Create Component
model=GaussianIMC(mol_list,temp,offset,std,angle,fwhm,gradient)
model.set_velocity(rad_vel)
univ.add_component('Methcloud',model)

# Defines a new component
mol_list = {'33SO2':[0.5,8.0]* u.Jy/u.beam}
offset   = [-20,20]*u.arcsec
std      = [4,12]*u.arcsec
angle    = np.pi/6.*u.rad
fwhm     = 10*u.km/u.s
gradient = [1.0,-1.0]*u.km/(u.s*u.arcsec)

# Create Component
model=GaussianIMC(mol_list,temp,offset,std,angle,fwhm,gradient)
model.set_velocity(rad_vel)
univ.add_component('Methcloud',model)


In [ ]:
# Create Cube
ang_res = [3.0,3.0]*u.arcsec
fov     = [200,200]*u.arcsec
freq    = 299.898*u.GHz
spe_res = 0.002*u.GHz
bw      = 0.2*u.GHz
noise   = 0.001*u.Jy/u.beam

cont = univ.gen_cube(center, ang_res, fov, freq, spe_res, bw, noise,noise/50.0)

## Denoise and Visualize

In [ ]:
synthetic=cont.primary
print "ORIGINAL M0"
visualize(moment0(synthetic))
syden=denoise(synthetic,threshold=2.0*estimate_rms(synthetic))
print "DENOISED M0"
visualize(moment0(syden))
print "DENOISED M1"
visualize(moment1(syden))
print "DENOISED M2"
visualize(moment2(syden))
#visualize(syden,contour=True)

## Metadata

In [ ]:
cont.tables[0]

In [ ]:
cont.tables[2]

# Algorithms

## ROI Detection

## Clump Detection (CUPID)

# What Next?

## More libraries!
* *2D imaging*     : Glue, AplPy ?
* *Dendograms*     : Astrodendro
* *Radioastronomy* : CASA (casacore)
* *(put your library here)*